In [12]:
import sys
import re
import pymongo
import json
import time
import datetime
import requests
from bs4 import BeautifulSoup

dbname = "fdac18mp2" #please use this database
collname1 = "glprj_mkramer6" #please modify so you store data in your collection
collname2 = "sfprj_mkramer6" #please modify so you store data in your collection
# beginning page index
begin = "0"
client = pymongo.MongoClient("da1.eecs.utk.edu")

db = client[dbname]
coll = db[collname1]
results = []

print("Start:")
beginurl = "https://gitlab.com/api/v4/projects?archived=false&membership=false&order_by=created_at&owned=false&page=" + begin + \
    "&per_page=99&simple=false&sort=desc&starred=false&statistics=false&with_custom_attributes=false&with_issues_enabled=false&with_merge_requests_enabled=false"

beginurl2 ="https://sourceforge.net/directory/?q=se&sort=name&page="

print('Note: list of projects at end.')
print('\n\nGitlabs:\n')

gleft = 0

header = {'per_page': 99}

# check remaining query chances for rate-limit restriction
def wait(left):
    global header
    while (left < 20):
        l = requests.get('https://gitlab.com/api/v4/projects', headers=header)
        if (l.ok):
            left = int(l.headers.get('RateLimit-Remaining'))
        time .sleep(60)
    return left

# send queries and extract urls 
def get(url, coll):
#def get(url):

    global gleft
    global header
    global bginnum
    gleft = wait(gleft)
    values = []
    size = 0
    prevcutURL = ''
    
    resIndex = 0;
    try:
        r = requests .get(url, headers=header)
        time .sleep(0.5)
        # got blocked
        if r.status_code == 403:
            return "got blocked", str(bginnum)
        if (r.ok):

            gleft = int(r.headers.get('RateLimit-Remaining'))
            lll = r.headers.get('Link')
            t = r.text
            array = json.loads(t)
            index = 0
            
            for el in array:
                if resIndex == 50:
                    return
                sChar = array[index]["name"][0]
                
                if sChar == 's':
                    
                    gitURL = array[index]["web_url"]
                    cutURL = gitURL.split('/')
                    if cutURL[3] != prevcutURL:
                        resURL1 = '{"web_url": "' + array[index]["web_url"] + '"}'
                        print(resURL1)
                        JsonURL1 = json.loads(resURL1)
                        #coll.insert_one(JsonURL1)
                        prevcutURL = cutURL[3]
                        resIndex = resIndex + 1
                        print('%d: ' % resIndex)
                        print('Owner: %s' % array[index]["name"])
                        print('Project: %s' % array[index]["namespace"]["name"])
                        print('URL: %s' % array[index]["web_url"])
                        print('\n')
                        results.append(array[index]["name"])
                    
                index = index + 1
 
            #next page
            while ('; rel="next"' in lll):
                gleft = int(r.headers.get('RateLimit-Remaining'))
                gleft = wait(gleft)
                # extract next page url
                ll = lll.replace(';', ',').split(',')
                url = ll[ll.index(' rel="next"') -
                         1].replace('<', '').replace('>', '').lstrip()
             
                try:
                    
                    r = requests .get(url, headers=header)
                    if r.status_code == 403:
                        return "got blocked", str(bginnum)
                    if (r.ok):
                        lll = r.headers.get('Link')
                        t = r.text
                        array1 = json.loads(t)
                        index1 = 0
                        for el in array1:
                            if resIndex == 50:
                                return
                            sChar = array1[index1]["name"][0]
                            if sChar == 's':
                                
                                gitURL = array1[index1]["web_url"]
                                cutURL = gitURL.split('/')
                                
                                if cutURL[3] != prevcutURL:
                                    resURL2 = '{"web_url": "' + array1[index1]["web_url"] + '"}'
                                    print(resURL2)
                                    JsonURL12 = json.loads(resURL2)
                                    #coll.insert_one(JsonURL12)
                                    resIndex = resIndex + 1
                                    print('%d: ' % resIndex)
                                    print('Owner: %s' % array[index1]["namespace"]["name"])
                                    print('Project: %s' % array1[index1]["name"])
                                    print('URL: %s' % array1[index1]["web_url"])
                                    prevcutURL = cutURL[3]
                                    print('\n')
                                    results.append(array1[index1]["name"])
                            index1 = index1 + 1
                    else:
                        sys.stderr.write("url can not found:\n" + url + '\n')
                        return 
                except requests.exceptions.ConnectionError:
                    sys.stderr.write('could not get ' + url + '\n')

        else:
            sys.stderr.write("url can not found:\n" + url + '\n')
            return

    except requests.exceptions.ConnectionError:
        sys.stderr.write('could not get ' + url + '\n')
    except Exception as e:
        sys.stderr.write(url + ';' + str(e) + '\n')
     

    
def get2(beginurl2, coll):
    
    curPage = 40
    index = 0
    ifEnd = False
    while True:
        url = beginurl2 + str(curPage)
        r = requests.get(url)
        page_content = BeautifulSoup(r.text, "html.parser")
        HeadEl = page_content.find_all(class_="result-heading-texts")

        for j in HeadEl:
            projURL = "https://sourceforge.net/"
            header = j.find_all("h2")
            l = 0
            result = ""
            tmp = ""
            head = str(header[0])
            tmp = head.replace("<h2>", "")
            result = tmp.replace("</h2>", "")

            if index == 50:
                ifEnd = True
                break
                
            if result[0] == 'S':
                index = index + 1
                print("%d:" % index)
                print("Project Name: %s" % result)

                pURL = j.find_all("a")
                tmp = ""
                tmp = str(pURL[0])

                result2 = ""
                sNum = 0
                for k in tmp:
                    if sNum > 9:
                        if k == chr(34):
                            break
                        result2 = result2 + k
                    sNum = sNum + 1

                projURL = projURL + result2
                print("URL: %s" % projURL)
                print("\n")
                projURLresult = '{"web_url": "' + projURL + '"}'
                JSONurl2 = json.loads(projURLresult)
                coll.insert_one(JSONurl2)
        
        if(ifEnd == True):
            break
            
        curPage = curPage + 1

    print("done")
    
#start retrieving
#collname = "glprj_mkramer6" #please modify so you store data in your collection
#get(beginurl,coll)

print('\n\nSourceForge:\n')

coll = db[collname2]
get2(beginurl2, coll)

Start:
Note: list of projects at end.


Gitlabs:



SourceForge:

1:
Project Name: SAAAD
URL: https://sourceforge.net/projects/saaad/


2:
Project Name: SAE - School Administration System
URL: https://sourceforge.net/projects/sae-admin/


3:
Project Name: SAGA
URL: https://sourceforge.net/projects/saga2008/


4:
Project Name: SAGA GIS BRASIL
URL: https://sourceforge.net/projects/saga-gis-br/


5:
Project Name: SAINP
URL: https://sourceforge.net/projects/sainpfinance/


6:
Project Name: SDK Server Matrix Shell
URL: https://sourceforge.net/projects/sdkservermatrix/


7:
Project Name: SE - Room booking system
URL: https://sourceforge.net/projects/roombookingsys/


8:
Project Name: SE 3000
URL: https://sourceforge.net/projects/se3000.arris/


9:
Project Name: SE Android
URL: https://sourceforge.net/projects/seandroid/


10:
Project Name: SE Auditor
URL: https://sourceforge.net/projects/seauditor/


11:
Project Name: SE Case Study File Manager
URL: https://sourceforge.net/projects/filemanag